In [1]:
# http://qwone.com/~jason/20Newsgroups/ 下载 20-news 数据
# 20-news 数据集包含 20 个不同主题
# 20-news 数据集的目录结构如下：
# 20news-18828
# ├── alt.atheism
# ├── comp.graphics
# ├── comp.os.ms-windows.misc
# ├── comp.sys.ibm.pc.hardware
# ├── comp.sys.mac.hardware
# ├── comp.windows.x
# ├── misc.forsale
# ├── rec.autos
# ├── rec.motorcycles
# ├── rec.sport.baseball
# ├── rec.sport.hockey
# ├── sci.crypt
# ├── sci.electronics
# ├── sci.med
# ├── sci.space
# ├── soc.religion.christian
# ├── talk.politics.guns
# ├── talk.politics.mideast
# ├── talk.politics.misc
# └── talk.religion.misc


from pyspark.sql import SparkSession
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

spark = SparkSession.builder.appName("lecture3").getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/13 21:43:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
from nltk.corpus import stopwords

# 试用前需要下载 nltk 的 stop_words
# import nltk
# nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

In [10]:
def remove_punctuation(text):
    # 删除标点符号
    import string
    return text.translate(str.maketrans('', '', string.punctuation))

In [11]:
remove_punctuation("Hi, I'm peter!")

'Hi Im peter'

In [12]:
rdd = sc.wholeTextFiles('20news-18828/comp.graphics/*')

In [13]:
# map step in mapreduce
# 转换成小写
rdd = rdd.map(lambda x: x[1].lower())
rdd.take(1)
# 分词
rdd = rdd.flatMap(lambda x: x.split())
rdd.take(1)
# 去除停用词
rdd = rdd.filter(lambda x: x not in stop_words)
# 去除标点符号
rdd = rdd.map(remove_punctuation)
# 去掉停用词和空字符串
rdd = rdd.filter(lambda x: x not in stop_words)
rdd = rdd.filter(lambda x: x != '')
# 每个词出现就计1
rdd = rdd.map(lambda x: (x, 1))
rdd.take(1)

["from: lipman@oasys.dt.navy.mil (robert lipman)\nsubject: call for presentations: navy sciviz/vr seminar\n\n\n\t\t\tcall for presentations\n\t\n      navy scientific visualization and virtual reality seminar\n\n\t\t\ttuesday, june 22, 1993\n\n\t    carderock division, naval surface warfare center\n\t      (formerly the david taylor research center)\n\t\t\t  bethesda, maryland\n\nsponsor: ness (navy engineering software system) is sponsoring a \none-day navy scientific visualization and virtual reality seminar.  \nthe purpose of the seminar is to present and exchange information for\nnavy-related scientific visualization and virtual reality programs, \nresearch, developments, and applications.\n\npresentations: presentations are solicited on all aspects of \nnavy-related scientific visualization and virtual reality.  all \ncurrent work, works-in-progress, and proposed work by navy \norganizations will be considered.  four types of presentations are \navailable.\n\n     1. regular prese

['from:']

[('lipmanoasysdtnavymil', 1)]

In [14]:
# reduce step in mapreduce
# 按照 key 进行聚合
rdd = rdd.reduceByKey(lambda x, y: x + y)
# 按照词频排序
rdd = rdd.sortBy(lambda x: x[1], ascending=False)
rdd.take(200)

[('image', 1203),
 ('subject', 1060),
 ('graphics', 755),
 ('jpeg', 693),
 ('file', 682),
 ('images', 582),
 ('would', 566),
 ('also', 539),
 ('files', 509),
 ('software', 507),
 ('use', 499),
 ('available', 487),
 ('one', 487),
 ('format', 476),
 ('data', 471),
 ('program', 447),
 ('get', 437),
 ('like', 433),
 ('know', 406),
 ('writes', 391),
 ('version', 383),
 ('email', 382),
 ('gif', 380),
 ('3d', 377),
 ('article', 366),
 ('computer', 360),
 ('ftp', 353),
 ('need', 353),
 ('color', 351),
 ('information', 341),
 ('please', 330),
 ('display', 321),
 ('help', 314),
 ('thanks', 306),
 ('im', 304),
 ('system', 304),
 ('see', 298),
 ('anyone', 295),
 ('package', 293),
 ('using', 283),
 ('1', 281),
 ('code', 281),
 ('bit', 274),
 ('2', 273),
 ('find', 273),
 ('may', 264),
 ('many', 262),
 ('etc', 255),
 ('could', 254),
 ('good', 250),
 ('free', 245),
 ('programs', 239),
 ('much', 239),
 ('send', 238),
 ('want', 238),
 ('c', 237),
 ('well', 230),
 ('time', 229),
 ('line', 225),
 ('number

In [47]:
# RDD 的 Join 操作

rdd1 = sc.parallelize([(1001, 'Mike'), (1002, 'Peter'), (1003, 'John'), (1004, 'Mary')])
rdd2 = sc.parallelize([(1001, 'Sales'), (1002, 'IT'), (1005, 'Engineering')])

rdd1.join(rdd2).collect()

rdd1.leftOuterJoin(rdd2).collect()

rdd1.rightOuterJoin(rdd2).collect()

[(1001, ('Mike', 'Sales')), (1002, ('Peter', 'IT'))]

[(1001, ('Mike', 'Sales')),
 (1002, ('Peter', 'IT')),
 (1003, ('John', None)),
 (1004, ('Mary', None))]

[(1001, ('Mike', 'Sales')),
 (1002, ('Peter', 'IT')),
 (1005, (None, 'Engineering'))]